<a id="top"></a>
# Question and Answering Model Scoring
*** 

**Note : BERT models are memory intensive, try with bigger environment for smoother performance (8vCPU and 16GB RAM works perfectly)**     

## Create Client Connections to Watson Machine Learning & Set Default Space

In this step we create the Watson Machine Learning client, get the deployment_space_uid, and set the default space.


In [1]:
# WML IMPORT: ACCESS DEPLOYED MODELS
import ibm_watson_machine_learning
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

# READING AND WRITING PROJECT ASSETS
import project_lib
import sys
project = project_lib.Project()

if '/project_data/data_asset/' not in sys.path:
    sys.path.insert(0, '/project_data/data_asset/')


### Provide the name for space, model and deployment
- Space name is the name given to a space, which is the containerized part of all assets that you push to the deployment section (Watson Machine Learning)
- Model name is the name you want to give to the model
- Deployment name is the name of the deployment of your model

In [2]:
# Specify a name for the space being created, the saved models and the model deployments
space_name = 'WML MSK Benefits'
model_name = 'wml_msk_benefits_model'
deployment_name = 'wml_msk_benefits_model_deployment'

use_existing_space=True

### Connect to the Watson Machine Learning client

In [3]:
from ibm_watson_machine_learning import APIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.5"
}

wml_client = APIClient(wml_credentials)

In [4]:
space_uid=""
for space in wml_client.spaces.get_details()['resources']:

    if space['entity']['name'] ==space_name:
        print("Deployment space with ",space_name,"already exists . .")
        space_uid=space['metadata']['id']
        wml_client.set.default_space(space_uid)
        print("Using the existing space")

Deployment space with  WML MSK Benefits already exists . .
Using the existing space


## Specify the deployment function parameters
These parameters will be used to find the depoyment for scoring 

In [5]:
header = {'Content-Type': 'application/json'}
ai_parms = {'wml_credentials' : wml_credentials,'header' : header}

### Define name for your function and the deployment of that function

In [6]:
FUNCTION_NAME = 'MSK_WD_Wrapper_Bert'
FUNCTION_DEPLOYMENT_NAME = 'msk_wd_pipeline_deployment_bert'

### Retrieve the function deployment ID for using with API

In [7]:
deployment_details = wml_client.deployments.get_details()
function_deployment_id = ''
function_id = ''
for deployment in deployment_details['resources']:
    f_deployment_id = deployment['metadata']['id']
    f_id = deployment['entity']['asset']['id']
    if deployment['entity']['name'] == FUNCTION_DEPLOYMENT_NAME:
        function_deployment_id = f_deployment_id
        function_id = f_id
        print('Found function deployment ID', f_deployment_id)
        print('Corresponding function ID', f_id)
        break

Found function deployment ID 0d768527-f882-411b-b154-79fa481ad5f3
Corresponding function ID fc65d340-186b-445e-a6e0-14030e0ae752


## Scoring 

### Internal scoring URL of the deployed function
To get the external scoring url, in case you want to call this url from outside the environment replace <internal-nginx-svc:port> with your CPD envrionment URL

In [8]:
meta_props_deployment = {
   wml_client.deployments.ConfigurationMetaNames.NAME: FUNCTION_DEPLOYMENT_NAME,
   wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: FUNCTION_DEPLOYMENT_NAME,
   wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "L", "num_nodes" : 20},

   wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_scoring_url = wml_client.deployments.get_details(function_deployment_id)['entity']['status']['online_url']['url']


In [9]:
print(function_scoring_url)

https://internal-nginx-svc:12443/ml/v4/deployments/0d768527-f882-411b-b154-79fa481ad5f3/predictions


### Function to generate an authorization token needed to call the deployed function

In [11]:
import requests
from requests.auth import HTTPBasicAuth

def generate_access_token(user = '', password = ''):
    if user =='':
    #Using the default token from the current session instead if the
    #same server is used: 
        return os.environ['USER_ACCESS_TOKEN']
    
    
    #otherwise using the provided credentials: 
    headers={}
    headers["Accept"] = "application/json"
    
    #WARNING : Replace with your username and password
    auth = HTTPBasicAuth(user, password)

    #WARNING : Replace with your CPD environment URL
    ICP_TOKEN_URL= "https://cpd-cpd-cpd.apps.testcp4.mskcc.org//v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    #access token is missing if authentication failed
    #next statement generates exception in case of authentication failure
    icp_access_token = json_data['accessToken']
    return icp_access_token

### Scoring with the deployment function

In [21]:
#Other examples: 
question_ ='What expenses are covered by vision plan?'
# question_ = "What is the customer care number for HR?" 

In [22]:
# NOTE: manually define and pass the array(s) of values to be scored
#payload_scoring = {"question" :  'Will I accrue vacation time while I am on leave?'}
request_data = {"values": question_, "fields": "question"}

payload = {"input_data": [request_data]}

### Scoring using WML client

In [23]:
%%time
predictions = wml_client.deployments.score(function_deployment_id, payload)
print(predictions)

{'predictions': [{'fields': ['answers'], 'values': [['health - vision', 'health - covered'], [{'score': 0.436814546585083, 'start': 89, 'end': 95, 'answer': 'vision', 'expanded_answer': 'You can use to pay eligible out-of-pocket healthcare (medical, prescription, dental, and vision) expenses now or in the future', 'context': 'You can use to pay eligible out-of-pocket healthcare (medical, prescription, dental, and vision) expenses now or in the future. You’re only eligible for an HSA if you’re enrolled in the UHC CDHP. If you enroll in a different medical plan, or waive medical coverage, the Health Care Flexible Spending Account (FSA) is a great alternative.', 'url': 'https://mskbenefits.mskcc.org/new-hires/tour'}, {'score': 0.35381725430488586, 'start': 34, 'end': 62, 'answer': 'eligible deductible expenses', 'expanded_answer': 'I am requesting reimbursement for eligible deductible expenses incurred by insurance covered a portion of the expense', 'context': 'I am requesting reimburseme

### Scoring using HTTP request/response directly

In [34]:
%%time
import urllib3, requests, json
import os

#If user/password should be used for authorization purposes
#please provide the user name and password as in this example: 
#token = generate_access_token('user', 'password')
token = generate_access_token()

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token}
response_scoring = requests.post(function_scoring_url+"?version=2021-08-21", json=payload, headers=header, verify=False)

print("Scoring response")
print(response_scoring.text)

Scoring response
{"predictions": [{"fields": ["answers"], "values": [["health - vision", "health - covered"], [{"score": 0.436814546585083, "start": 89, "end": 95, "answer": "vision", "expanded_answer": "You can use to pay eligible out-of-pocket healthcare (medical, prescription, dental, and vision) expenses now or in the future", "context": "You can use to pay eligible out-of-pocket healthcare (medical, prescription, dental, and vision) expenses now or in the future. You\u2019re only eligible for an HSA if you\u2019re enrolled in the UHC CDHP. If you enroll in a different medical plan, or waive medical coverage, the Health Care Flexible Spending Account (FSA) is a great alternative.", "url": "https://mskbenefits.mskcc.org/new-hires/tour"}, {"score": 0.35381725430488586, "start": 34, "end": 62, "answer": "eligible deductible expenses", "expanded_answer": "I am requesting reimbursement for eligible deductible expenses incurred by insurance covered a portion of the expense", "context": "

[Back to top](#top)